In [ ]:
#Import usefull libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import datetime

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [ ]:
#Import data
data = pd.read_csv(r'/Users/edgartome_1/IronHack/IronProjects/Project8/Data/uber-raw-data-apr14.csv')

#Overview
data.head()

In [ ]:
#Printing data infos: data types, missimng values, shape
#The dataframe is composed of 4 columns and 564516 rows, no missing values

data.info()

In [ ]:
#We have to change the Date/Time data type

data['Date/Time'] = pd.to_datetime(data['Date/Time'], format='%m/%d/%Y %H:%M:%S')

In [ ]:
#Checking result

data.head(5)

In [ ]:
#Checking result

data.dtypes

In [ ]:
#Extract hour from datetime

data['Hour'] = pd.to_datetime(data['Date/Time']).dt.hour

In [ ]:
#Extract day

data['Day'] = data['Date/Time'].dt.dayofweek

In [ ]:
#Making sure columns has been created has intended

data.head()

In [ ]:
#View the days

data.Day.value_counts()

In [ ]:
#View of the hours

data.Hour.value_counts()

In [ ]:
#Encondig the Base colum of data frame

from sklearn.preprocessing import LabelEncoder

tst = data.copy()

le = LabelEncoder()

data['Base'] = le.fit_transform(data.Base.values)

In [ ]:
#View the types of columns

data.dtypes

In [ ]:
#Checking values of Base column
#We can see there are 5 Bases

data.Base.value_counts()

## EDA

In [ ]:
#Check repartition of the car, coordonates speaking

sns.boxplot(data=data.Lat)

In [ ]:
#Check repartition of the car, coordonates speaking

sns.boxplot(data=data.Lon)

In [ ]:
#data.to_csv(r'/uber_cleaned.csv')

## Kmean clustering

In [ ]:
#Take a sample of 10%
data_sample = data.sample(int(data.shape[0]*0.1))
data_sample.shape

In [ ]:
#Drop the datetime column as the dtypes bring trouble

data_sample = data_sample.drop(columns='Date/Time')

In [ ]:
data_sample.dtypes

In [ ]:
#Scaling data

from sklearn.preprocessing import StandardScaler

x = data_sample[['Lat', 'Lon']]  #sample because it will allow to do faster code

scaler = StandardScaler()

print(scaler.fit(x))
print(scaler.mean_)

uber_scale = scaler.transform(x)
print(uber_scale)

In [ ]:
#Defining right amount of cluster using Elbow method
#5 clusters with kmeans

from yellowbrick.cluster import KElbowVisualizer

model = KMeans()

visualizer = KElbowVisualizer(estimator = model, k = (2,10))

visualizer.fit(x)

visualizer.poof()

In [ ]:
#Create model / predict

model = KMeans(5)

y_kmeans = model.fit_predict(x)

In [ ]:
np.unique(y_kmeans, return_counts=True)

In [ ]:
#Adding label to df
data_sample['labels_kmeans'] = model.labels_


In [ ]:
#Plotting

sns.scatterplot(data=data_sample, x='Lon', y='Lat', hue='labels_kmeans', palette='deep')
plt.show()

## DBSCAN clustering

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
#Selecting hyperparameters for dbscan

def get_kdist_plot(X=None, k=None, radius_nbrs=1.0):

    nbrs = NearestNeighbors(n_neighbors=k, radius=radius_nbrs).fit(x)

    # For each point, compute distances to its k-nearest neighbors
    distances, indices = nbrs.kneighbors(x) 
                                       
    distances = np.sort(distances, axis=0)
    distances = distances[:, k-1]

    # Plot the sorted K-nearest neighbor distance for each point in the dataset
    plt.figure(figsize=(8,8))
    plt.plot(distances)
    plt.xlabel('Points/Objects in the dataset', fontsize=12)
    plt.ylabel('Sorted {}-nearest neighbor distance'.format(k), fontsize=12)
    plt.grid(True, linestyle="--", color='black', alpha=0.4)
    plt.show()
    plt.close()


k = 2 * x.shape[-1] - 1 # k=2*{dim(dataset)} - 1
get_kdist_plot(X=x, k=k)

In [ ]:
#DBSCAN clustering
#We can see in previous graph that eps is more or less 0.01

dbscan = DBSCAN(eps=0.01, min_samples=20, n_jobs=-1)

dbscan.fit(x)

y_dbscan = dbscan.fit_predict(x)

In [ ]:
#Adding label to df

data_sample['labels_dbscan'] = dbscan.labels_

In [ ]:
#Plotting

sns.scatterplot(data=data_sample, x='Lon', y='Lat', hue='labels_dbscan', palette='deep')
plt.show()

## Agglomerative clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

In [ ]:
#To defined the numer of clusters

dendrogram = sch.dendrogram(sch.linkage(x, method='ward'))

In [ ]:
#Agglomerative clustering
#We can see in previous graph that number of clusters is 5

clustering = AgglomerativeClustering(n_clusters=5).fit(x)

y_agg = clustering.fit_predict(x)

#Adding label to df

data_sample['labels_agg'] = clustering.labels_

In [ ]:
#Plotting
sns.scatterplot(data=data_sample, x='Lon', y='Lat', hue='labels_agg', palette='deep')
plt.show()

In [ ]:
data_sample.to_csv(r'/Users/edgartome_1/IronHack/IronProjects/Project8/Data/uber_cleaned_clusters_edgar.csv')

## Comparison

In [ ]:
from sklearn import metrics
print('KMeans')
print('=============')
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(data_sample,data_sample['labels_kmeans'],  metric = 'sqeuclidean'))
print("calinski_harabasz: %0.3f" % metrics.calinski_harabasz_score(data_sample,data_sample['labels_kmeans'] ))

print('_____________')
print('DBSCAN')
print('=============')
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(data_sample,data_sample['labels_dbscan'],  metric = 'sqeuclidean'))
print("calinski_harabasz: %0.3f" % metrics.calinski_harabasz_score(data_sample,data_sample['labels_dbscan'] ))

print('_____________')
print('Agglomerative')
print('=============')
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(data_sample,data_sample['labels_agg'],  metric = 'sqeuclidean'))
print("calinski_harabasz: %0.3f" % metrics.calinski_harabasz_score(data_sample,data_sample['labels_agg'] ))

## Map

In [ ]:
data_sample = pd.read_csv(r'/Users/edgartome_1/IronHack/IronProjects/Project8/Data/uber_cleaned_clusters_edgar.csv')

In [ ]:
#Library to plot map
import folium

In [ ]:
data_sample.head()

In [ ]:
def map(data, cluster_column, Day):
    m = folium.Map(location=[data.Lat.mean(), data.Lon.mean()], zoom_start=9, tiles='OpenStreet Map')
    for _, row in data.iterrows():
        if row[cluster_column] == -1 and row[Day]:
            cluster_colour = '#006400'
        elif row[cluster_column] == 0 and row[Day]:
            cluster_colour = '#FF5233'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
        elif row[cluster_column] == 1 and row[Day]:
            cluster_colour = '#641E16'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
        elif row[cluster_column] == 2 and row[Day]:
            cluster_colour = '#7AFF33'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
        elif row[cluster_column] == 3 and row[Day]:
            cluster_colour = '#33FFF9'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
        elif row[cluster_column] == 4 and row[Day]:
            cluster_colour = '#6133FF'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
        elif row[cluster_column] == 5 and row[Day]:
            cluster_colour = '#FF33FC'
            folium.CircleMarker(
                location= [row['Lat'], row['Lon']],
                radius=1,
                popup= row[cluster_column],
                color=cluster_colour,
                fill=True,
                fill_color=cluster_colour
            ).add_to(m)
    return m 

In [ ]:
#Day from 0 to 6
#Map result on April, Thursday for Model of Agglomerative clustering

map(data_sample, 'labels_agg', 3) #Day 3

In [ ]:
#Day from 0 to 6
#Map result on April, Thursday for Model of DBSCAN clustering

map(data_sample, 'labels_dbscan', 3) #Day 3

In [ ]:
#Day from 0 to 6
#Map result on April, Thursday for Model of Kmenans clustering

map(data_sample, 'labels_kmeans', 3) #Day 3

## Best Model
### Model of Agglomerative clustering

#### Acording with the visualizations and the values of Silhouette Coefficient: 0.019 calinski_harabasz: 477.214

## Running September data with the model

In [ ]:
#Import
sept = pd.read_csv(r'/Users/edgartome_1/IronHack/IronProjects/Project8/Data/uber-raw-data-sep14.csv')

In [ ]:
sept.head()

In [ ]:
#Clean data like we clean the first dataset

sept['Date/Time'] = pd.to_datetime(sept['Date/Time'], format='%m/%d/%Y %H:%M:%S')

In [ ]:
#Create column with hour

sept['Hour'] = pd.to_datetime(sept['Date/Time']).dt.hour

In [ ]:
#Create column with hour

sept['Day'] = sept['Date/Time'].dt.dayofweek

In [ ]:
#To encode the Base column
from sklearn.preprocessing import LabelEncoder
tst = sept.copy()
le = LabelEncoder()
sept['Base'] = le.fit_transform(sept.Base.values)

In [ ]:
#To transform the data in same size as the sample

sept_sample = sept[:56451]
sept_sample.shape

In [ ]:
#Drop column od Date/Time tahe is already on colum day and hour

sept_sample = sept_sample.drop(columns='Date/Time')

In [ ]:
sept_sample.head()

In [ ]:
sept_sample['labels_agg'] = clustering.labels_

In [ ]:
#Map result on September, Thursday
map(sept_sample, 'labels_agg', 3)